In [1]:
from spacy.lang.en import English
from common import create_tops_patterns
import spacy

Init Plugin
Init Graph Optimizer
Init Kernel


In [25]:
create_tops_patterns()

[[{'LOWER': 't'}, {'IS_PUNCT': True, 'OP': '?'}, {'LOWER': 'shirt'}],
 [{'LEMMA': {'IN': ['jacket',
     'camisole',
     'shirt',
     'coat',
     'sweater',
     'blouse',
     'kimono',
     'cardigan',
     'hoodie',
     'vest',
     'poncho',
     'blazer',
     'sweatshirt',
     '2aistcoat',
     'bralette',
     'bra',
     'jersey',
     't',
     'tee',
     'tank',
     'crop',
     'croptee',
     'croptop',
     'tanktop',
     'top',
     'coverup']},
   'POS': {'NOT_IN': ['ADJ']}}],
 [{'LEMMA': 'tank'}, {'IS_PUNCT': True, 'OP': '?'}, {'LEMMA': 'top'}]]

In [3]:
'''
#old_version spaCy 2.0 and not usable anymore
nlp = English()
ruler = EntityRuler(nlp)
ruler.add_patterns([{'label':'TOPS', 'pattern': p} for p in create_tops_patterns()])
nlp.add_pipe(ruler)
'''


"\n#old_version spaCy 2.0 and not usable anymore\nnlp = English()\nruler = EntityRuler(nlp)\nruler.add_patterns([{'label':'TOPS', 'pattern': p} for p in create_tops_patterns()])\nnlp.add_pipe(ruler)\n"

In [50]:
#initilize empty pipeline model
# nlp = English()
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.blank("en")
#intilize EntityRuler
entity_ruler_TOPS = nlp.add_pipe("entity_ruler")


In [51]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x17b637040>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x17c7f9340>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x17bb0aeb0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x17c7fb6c0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x17c95a4c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x17c9db040>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x17b7b55c0>)]

In [52]:
nlp.remove_pipe('ner')


('ner', <spacy.pipeline.ner.EntityRecognizer at 0x17c9db040>)

In [53]:
entity_ruler_TOPS.add_patterns([{'label':'TOPS', 'pattern': p} for p in create_tops_patterns()])

In [54]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x17b637040>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x17c7f9340>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x17bb0aeb0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x17c7fb6c0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x17c95a4c0>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x17b7b55c0>)]

In [55]:
import common as c
s = c.clean_product_description("The Tempo Hoodie TOPS is the UPF 50+ activewear you've been looking for! It has thumbholes, a kangaroo pocket, and a hood for when the sun is too hot or you forgot your hat. Our Fitness Hoodie TOPS is made out of our Active Athlon fabric with the added bonus of our Cooltect™ technology. You can be active in this fitted Fitness Hoodie TOPS without getting uncomfortably hot. So go ahead and enjoy sun-safe biking, walking, running and so much more!Highlights:UPF 50+Raglan long sleeves with thumbholesWelt kangaroo pocketHoodedActive Athlon™ fabric: Lightweight and breathable with moisture wicking for quick dry performanceCooltect™ technology accelerates moisture wicking to keep you cooler and more comfortable")

doc = nlp(s)
print([(ent.text, ent.label_) for ent in doc.ents])

[('hoodie', 'TOPS'), ('hoodie', 'TOPS'), ('tops', 'TOPS'), ('hoodie', 'TOPS'), ('tops', 'TOPS')]


In [56]:
doc.ents

(hoodie, hoodie, tops, hoodie, tops)

In [48]:
from spacy import displacy
displacy.render(doc, style='ent')
displacy.render(doc)

/Users/luis/miniforge3/envs/tensorflow/lib/python3.9/site-packages/spacy/displacy/__init__.py:133: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


Save the entity_ruler for 'TOPS' to disk

In [57]:
entity_ruler_TOPS.to_disk("entity_ruler_TOPS.jsonl")
nlp.to_disk("rule_model_TOPS")

Preparing Data for Scoring
Need some data to compare both method.(certain format)

In [63]:
from spacy.training import docs_to_json
from spacy.training.iob_utils import biluo_tags_from_offsets, spans_from_biluo_tags
import srsly


In [58]:
#for parsing data to TRIAN_DATA purpose
from spacy.matcher import Matcher

tops_matcher = Matcher(nlp.vocab)
#This rule_based matcher is only to detect "TOPS"
tops_matcher.add("TOPS_TYPE", create_tops_patterns())

def parse_train_data_forDB(text):
    doc = nlp(text)
    #ignore for now 
    #detections = [(doc[start:end].start_char, doc[start:end].end_char, 'TOPS') for idx, start, end in type_matcher(doc) ]
    
    spans = [doc[start:end] for _, start, end in tops_matcher(doc)]
    detections =  [(span.start_char, span.end_char, 'TOPS') for span in spacy.util.filter_spans(spans)] #remove duplicates or overlaps using spacy.util.filter_spans
    
    return (doc.text, detections)

In [83]:
import pandas as pd



In [43]:
'''Example for docBin trainning format
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en")
training_data = [
  ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
]
# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    ents = []
    
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        print(span)
        ents.append(span)
        print(ents)
        
    doc.ents = ents
    db.add(doc)
  #db.to_disk("./train.spacy")  
  '''

Tokyo Tower
[Tokyo Tower]


In [60]:
import common as c
title_tags_type_df = tops_test_data[['title',  'product_type', 'tags']]
title_tags_type_df.insert(len(title_tags_type_df.columns), 'raw_combined_text', '')
for i in range(len(title_tags_type_df)):
    try:
        raw_combined_text = c.clean_tags_text(title_tags_type_df.loc[i, 'title'], title_tags_type_df.loc[i, 'product_type'], title_tags_type_df.loc[i, 'tags'])
        
        title_tags_type_df.loc[i, 'raw_combined_text'] = raw_combined_text
    except:
        print("something wrong in line# : ", i)

In [61]:
TRAIN_DATA  = [parse_train_data_forDB(d) for d in nlp.pipe(title_tags_type_df.loc[:,'raw_combined_text'])]

In [65]:

docs = []

for text, annot in TRAIN_DATA:
    
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot)
    entities = spans_from_biluo_tags(doc, tags)
    doc.set_ents(entities)
    docs.append(doc)
    

#srsly.write_json("spacy_format.json", docs_to_json(docs))

In [66]:
from spacy.tokens import Doc, DocBin
docs_bin = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])
for text, annot in TRAIN_DATA[0:1]:
    print(text)
    print()
    print(annot)
    


the gift sports bra, sports bra, galaxy nu psychedelic sacred geometry space

[(16, 19, 'TOPS'), (28, 31, 'TOPS')]


In [67]:
for text, annot in TRAIN_DATA[0:1]:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot)
    entities = spans_from_biluo_tags(doc, tags)
    
    print(entities)

[bra, bra]


In [68]:
nlp_stat = spacy.load("ML_based_model")
nlp_rule = spacy.load("rule_model_TOPS")

In [69]:
from spacy import displacy
txt1 = c.clean_product_description("SweetLegs is excited to introduce Funky Bunch Plus SweetLegs, a bold and electrifying addition to our 2021 Plus Size leggings collection! This black-based print features a bright neon geometric pattern that wouldn't be out of place in a 90's party! It's perfect for those who like to make an entrance.Funky Bunch Plus Size Leggings look rad styled with a relaxed 90's-inspired denim jacket, a SweetTops Classic Tee in white, and a pair of orange converse.")

displacy.render(nlp_stat(txt1), style="ent")
displacy.render(nlp_rule(txt1), style="ent")



In [70]:

txt2 = "This item is already made and ships out within 5 business days.Out of your size? Don't worry, order HERE100% soft-spun premium polyesterVibrant all over front &amp; back print Fade, crack &amp; wrinkle resistantLight fleece liningOur sweaters are made with love one at a time,Cut, sewn &amp; hand finished.. just for you!"
txt2 = c.clean_product_description(txt2)
displacy.render(nlp_stat(txt2), style="ent")
displacy.render(nlp_rule(txt2), style="ent")

In [71]:

txt3 = "Style #85143 – 08/0827Cozy up in this beautiful cable knit sweater! Made with recycled cotton, this sweater features a funnel neck, long sleeves and chunky knit. It is a timeless classic that will pair with everything in your closet for years to come.Style: PulloverFit: RelaxedLength: Slightly below hipsFunnel necklineLong sleeves with ribbed knit cuffsRibbed knit hemMachine wash in cold waterComposition: 75% Recycled Cotton, 25% Polyester"
txt3 = c.clean_product_description(txt3)
displacy.render(nlp_stat(txt3), style="ent")
displacy.render(nlp_rule(txt3), style="ent")

In [72]:
txt4 = "I wear a fancy T-SHirt TOPS and I got another button-down wonderful crop TOPS tee TOPS . Long shirt TOPS and coat TOPS are necessary for keeping warm in winter. Sweater TOPS and blouse TOPS are important for people living in the north. UA students have their own hoodies TOPS . The Tank Top TOPS is new stylish top-clothes. What about trying our new camisole which is fantastic?"
txt4 = c.clean_product_description(txt4)
displacy.render(nlp_stat(txt4), style="ent")
displacy.render(nlp_rule(txt4), style="ent")

In [79]:
txt5 = "The Tempo Hoodie TOPS is the UPF 50+ activewear you've been looking for! It has thumbholes, a kangaroo pocket, and a hood for when the sun is too hot or you forgot your hat. Our Fitness Hoodie TOPS is made out of our Active Athlon fabric with the added bonus of our Cooltect™ technology. You can be active in this fitted Fitness Hoodie TOPS without getting uncomfortably hot. So go ahead and enjoy sun-safe biking, walking, running and so much more!Highlights:UPF 50+Raglan long sleeves with thumbholesWelt kangaroo pocketHoodedActive Athlon™ fabric: Lightweight and breathable with moisture wicking for quick dry performanceCooltect™ technology accelerates moisture wicking to keep you cooler and more comfortable"
txt5 = c.clean_product_description(txt5)
#displacy.render(nlp_stat(txt5), style="ent")
#displacy.render(nlp_rule(txt5), style="ent")
print(len(nlp_stat(txt5).ents))
print(len(nlp_rule(txt5).ents))

7
5


### Based on Test_tops.csv to build ML and RM matcher.

In [97]:
tops_test_data = pd.read_csv("test_data/tops_test.csv")
tops_test_data.insert(len(tops_test_data.columns), 'label', 1, allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), 'nlp_stat_found', 0, allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), 'nlp_rule_found', 0, allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), 'nlp_stat_ents', "", allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), 'nlp_rule_ents', "", allow_duplicates=True)

tops_test_data.insert(len(tops_test_data.columns), '(description)nlp_stat_found', 0, allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), '(description)nlp_rule_found', 0, allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), '(description)nlp_stat_ents', "", allow_duplicates=True)
tops_test_data.insert(len(tops_test_data.columns), '(description)nlp_rule_ents', "", allow_duplicates=True)



tops_test_data.head()

,title,product_type,tags,body_html,label,nlp_stat_found,nlp_rule_found,nlp_stat_ents,nlp_rule_ents,(description)nlp_stat_found,(description)nlp_rule_found,(description)nlp_stat_ents,(description)nlp_rule_ents
0,THE GIFT SPORTS BRA,sports bra,"{Galaxy,Nu,Psychedelic,""Sacred Geometry"",Space}",Vibrant all over front &amp; back design100% p...,1,0,0,,,0,0,,
1,TEE SHIRT - BURNT ORANGE,tee shirt,"{fw20,gastown,""jack jones"",long,mens,ochre,ora...",PRODUCT DESCRIPTIONOne of the best selling lon...,1,0,0,,,0,0,,
2,DISAMBIGUATION DROP TEE,drop tee,"{Nu,Psychedelic,""Sacred Geometry""}",Vibrant all over front &amp; back design100% p...,1,0,0,,,0,0,,
3,Geometric Print Tunic Top - NEW ARRIVAL,tunic,"{221266,""JOSEPH RIBKOFF"",""Regular Price"",RESOR...","Style # 221266 - 03/0900This stretchy, silky k...",1,0,0,,,0,0,,
4,Billabong Arch S/S Tee,mens tees,"{20-40,25-50,availability_in-stock,billabong,c...",The Billabong Arch Short Sleeve T-Shirt featur...,1,0,0,,,0,0,,


In [76]:
title_tags_type_df.head()#this contains 'raw_combined_text' after cleaning title + product_type+ tags

,title,product_type,tags,raw_combined_text
0,THE GIFT SPORTS BRA,sports bra,"{Galaxy,Nu,Psychedelic,""Sacred Geometry"",Space}","the gift sports bra, sports bra, galaxy nu psy..."
1,TEE SHIRT - BURNT ORANGE,tee shirt,"{fw20,gastown,""jack jones"",long,mens,ochre,ora...","tee shirt - burnt orange, tee shirt, fw20 gast..."
2,DISAMBIGUATION DROP TEE,drop tee,"{Nu,Psychedelic,""Sacred Geometry""}","disambiguation drop tee, drop tee, nu psychede..."
3,Geometric Print Tunic Top - NEW ARRIVAL,tunic,"{221266,""JOSEPH RIBKOFF"",""Regular Price"",RESOR...","geometric print tunic top - new arrival, tunic..."
4,Billabong Arch S/S Tee,mens tees,"{20-40,25-50,availability_in-stock,billabong,c...","billabong arch s/s tee, mens tees, 20-40 25-50..."


In [98]:
for i in range(tops_test_data.shape[0]):
    try:
        s = title_tags_type_df.loc[i, 'raw_combined_text']
        tops_test_data.loc[i, "nlp_stat_found"] = len(nlp_stat(s).ents)
        tops_test_data.loc[i, "nlp_rule_found"] = len(nlp_rule(s).ents)
        tops_test_data.loc[i, "nlp_stat_ents"] = str(nlp_stat(s).ents)
        tops_test_data.loc[i, "nlp_rule_ents"] = str(nlp_rule(s).ents)
    except:
        print("Error: " + str(i) + s)
    

### Based on Title_Tags_Product_type two method comparasion 

In [100]:
rows = tops_test_data[tops_test_data.columns[0]].count()
print("total data sample: ", rows)
nlp_stat_found = tops_test_data.loc[tops_test_data['nlp_stat_found'] > 0].shape[0]
nlp_rule_found = tops_test_data.loc[tops_test_data['nlp_rule_found'] > 0].shape[0] 
based_on_ML_model = nlp_stat_found / rows
based_on_RB_model = nlp_rule_found / rows
print("based_on_ML_model: ", based_on_ML_model)
print("nlp_stat_found: ", nlp_stat_found, "/", rows)
print("based_on_RB_model: ",  based_on_RB_model)
print("nlp_stat_found: ", nlp_rule_found, "/", rows)

total data sample:  41
based_on_ML_model:  0.926829268292683
nlp_stat_found:  38 / 41
based_on_RB_model:  1.0
nlp_stat_found:  41 / 41


### Based on Product Description (body_html) two Method Comparasion 

In [106]:
for i in range(tops_test_data.shape[0]):
    try:
        s = tops_test_data.loc[i, 'body_html']
        s = c.clean_product_description(s)
       
        tops_test_data.loc[i, "(description)nlp_stat_found"] = len(nlp_stat(s).ents)
        tops_test_data.loc[i, "(description)nlp_rule_found"] = len(nlp_rule(s).ents)
        tops_test_data.loc[i, "(description)nlp_stat_ents"] = str(nlp_stat(s).ents)
        tops_test_data.loc[i, "(description)nlp_rule_ents"] = str(nlp_rule(s).ents)
    except:
        print("Error: " + str(i) + " " + s)
    

In [110]:
print("total data sample: ", rows)
bt_nlp_stat_found = tops_test_data.loc[tops_test_data['(description)nlp_stat_found'] > 0].shape[0]
bt_nlp_rule_found = tops_test_data.loc[tops_test_data['(description)nlp_rule_found'] > 0].shape[0] 
bt_based_on_ML_model = bt_nlp_stat_found / rows
bt_based_on_RB_model = bt_nlp_rule_found / rows
print("body_html_based_on_ML_model: ", bt_based_on_ML_model)
print("nlp_stat_found: ", bt_nlp_stat_found, "/", rows)
print("body_html_based_on_RB_model: ",  bt_based_on_RB_model)
print("nlp_stat_found: ", bt_nlp_rule_found, "/", rows)

total data sample:  41
body_html_based_on_ML_model:  0.36585365853658536
nlp_stat_found:  15 / 41
body_html_based_on_RB_model:  0.36585365853658536
nlp_stat_found:  15 / 41


In [107]:
tops_test_data.to_csv("test_data/valid_output.csv")